# Семинар 6. Сегментация изображений

1. Сегментация = попиксельная классификация
2. Требует меньшего объема тренировочных данных, чем классификация (пара сотен - пара тысяч картинок)
3. Только свёртки, нет fc-слоёв → размер изображения любой
4. Если для классификации нам нужны были картинки и их классы, то для сегментации нам нужны картинки и их маски (классы для каждого пикселя изображения).

![](images/1.png)

Пример маски:

![](images/3.png)

Из изображения размера `3xHxW` получаем изображение размера `CxHxW` при помощи сети. Дальше на каждый пиксель можем накинуть `softmax` (или `sigmoid`, если у нас бинарная сегментация) и получить предсказанные сетью вероятности. То есть в сегментации на вход кросс-энтропии идет не выход fc-слоя, а выход conv-слоя

![](images/4.png)

# Метрики и лосс для сегментации

## Метрики

**Accuracy** - имеет те же недостатки, что и для обычной классификации

**IoU** (intersection over union), или Jaccard index - самая распространенная

Наглядная иллюстрация IOU:
![](images/2.png)

$IoU = \frac{|X∩Y|}{|X∪Y|}$,
`X` - это gt-маска, а `Y` - это маска, предсказанная сетью

Например, в случае бинарной сегментации (1 - объект, 0 - фон) мы можем получить предсказанную сетью маску так: `y_mask = (y_pred > 0.5).float()`, здесь `y_pred` - это выход сигмоиды (то есть предсказанная сетью вероятность), а `0.5` - это наш порог для предсказания.
  
*Упражнение 1*: Выразите формулу $IOU$ через маски `y_pred` и `y_gt`

*Упражнение 2*: Зависит ли IOU от размера сегментируемого объекта? Обоснуйте ответ

*Упражнение 3*: Посмотрите на картинку, иллюстрирующую IOU. Один из квадратиков (левый) является gt, а второй - предсказанием. Какая часть картинки является `TP`, `FP`, `TN` и `FN`?
  
**DICE**

$DICE = \frac{2\times|X∩Y|}{|X|+|Y|}$

*Упражнение 1*: Выразите формулу $DICE$ через маски `y_pred` и `y_gt`

*Упражнение 2*: Выразите $DICE$ через $IOU$

*Упражнение 3*: Выразите $DICE$ через `precision` и `recall`

*Упражнение 4*: У $IOU$ есть наглядная интерпретация (см. картинку). Как интерпретировать $DICE$? Почему у него такая странная формула?

## Лоссы

- **Кросс-энтропия** - Подходит для задач классификации, и значит подойдёт для попиксельной классификации
- **Jaccard loss** - Похож на $1 - IOU$; минимизируя его, мы явно максимизируем $IOU$
- **Dice loss** - Похож на $1 - DICE$; минимизируя его, мы явно максимизируем $DICE$

*Упражнение 1*: Какая из этих метрик лучше подходит для задачи сегментации для несбалансированных классов?

*Упражнение 2*: Диффиренцируема ли метрика $IOU$? Если да, то посчитайте её производную. Если нет, то как почему можно использовать Jaccard loss в качестве функции потерь?

# Сети для сегментации. Upsampling

- В обычной классификационной сети у нас есть куча свёрток. После сверток карта признаков уменьшается в размере, но у нее становится больше каналов. В конце сети после свёрток есть что-нибудь вроде GAP, схлопывающего пространственные измерения, и fc-слой, переводящий признаки в классы.
- Уберем из сети GAP и fc-слои дальше, а оставим только последнюю карту признаков. В этом тензоре уже содержится достаточно много информации об объекте (где примерно он расположен и что примерно из себя представляет)
- Можем добавить upsample-слои, которые извлеченные из картинки фичи (малого `hxw` и с большим `c`) переведут в изображение исходного размера
- Получится fully convolutional сеть (без fc-слоёв)

![](images/5.png)

FCN = Efficient Sliding Window

Можно себе представить сегментацию как обычную классификацию. Представим, что у нас есть скользящее окно. Мы скользим этим окном по картинке, делаем кроп и предсказываем класс центрального пикселя. Но это жутко неэффективно, потому что одни и те же пиксели изображения будут участвовать в свертках при близких кропах.

*Иллюстрация справа:*

*1. Делаем предсказание для картинки 14x14, пропуская тензор через серию свёрток и нелинейностей - получаем карту признаков размера 1x1*

*2. Делаем предсказание для картинки 16x16 таким же образом - получаем карту признаков 4x4. Если бы мы прошлись скользящим окном размера 14x14 по картинке 16x16, то получили бы эти же 4 пикселя*

![](images/6.png)

*Картинка из лекций sim0nsays*

## Upsampling

Какие есть способы увеличить карту признаков в размере:

- Resize Nearest Neighbors
- MaxUnpooling
- Resize Bilinear
- **Transposed convolution** - самая распространенная, потому что есть обучаемые параметры, а значит сеть может выучить какие-то закономерности

![](images/7.png)

Можем увеличить карту признаков сразу до нужного размера за один раз (один `upsample`), а можем итеративно, сделав много `ConvTranspose` слоёв и нелинейностей между ними. Тогда границы будут определяться чётче

![](images/8.png)

## FCN

В нашей сети 5 страйдов x2, изображение в конце уменьшается в 32 раза. На последнем слое информация о классе очень высокоуровневая, но пространственные границы размыты. На более близком к началу сети уровне фичи более простые, но информация о пространственной координате еще не успела испортиться.

- Собираем информацию с последнего слоя pool5, увеличиваем в x32 раза (FCN-32s)

- Делаем `upsample` в 2 раза информации из pool5 и фьюзим (суммируем) с pool4 - можно  увеличить в x16 и будет еще одно предсказание (FCN-16s)

и т.д.

![](images/9.png)

![](images/10.png)

## UNet

- Чем более глубокий слой для `upsampling`'а мы используем, тем более качественные там фичи, но в то же время и хуже сохранилась пространственная информация (сравните `FCN-32s` и `FCN-8s` на картинке выше).
- А давайте будем прокидывать skip connection из энкодера в декодер. Благодаря этому нам будет передаваться пространственный контекст. Тут уже не суммирование после skip connection, как в `ResNet`, а конкатенация. После конкатенации есть свёртка, что позволяет выбирать сети, как перемешивать каналы. Можно и просто складывать (есть такая модификация архитектуры)
- Skip connection - это всегда хорошо для градиента
- `UNet` - это семейство архитектур. [Можно выбрать](https://github.com/qubvel/segmentation_models.pytorch) UNet со своим энкодером (например, `MobileNetV2`)

[Пост про UNet на Towards Data Science](https://towardsdatascience.com/u-net-b229b32b4a71)

![](images/11.png)

*Схема оригинального UNet*

### Feature Pyramid Network

Feaute Pyramid Network (FPN) очень похожа на UNet. Главное отличие в том, что в FPN предсказания делаются на разных масштабах, а потом объединяются. В UNet предсказание делается один раз в конце. То есть даже при одинаковой U-образной архитектуре, в FPN больше свёрток, т.е. эффективно больше глубина

![](images/12.png)

Feature pyramid часто применяется в сегментации и детекции

# Разное

Несколько моментов, которые еще не упомянули:

- Если используете аугментации, то важно, чтобы они были были одинаковыми и для маски, и для изображения (можно использовать `albumentations`)

- Два типа сегментации:
  - semantic segmentation
  - instance segmentation
  (см рисунок ниже)

  До сих пор мы говорили только о semantic segmentation, для которой два разных объекта одного класса неразличимы. Instance segmentation можно условно представлять себе как детекцию и сегментацию внутри бокса, который вернул детектор

![](images/13.png)

Примеры готовых репозиториев для сегментации:

- [segmentation models pytorch](https://github.com/qubvel/segmentation_models.pytorch)
- [MSegmentation](https://github.com/open-mmlab/mmsegmentation)